### This script requires a .csv file containing the minimum distances for valid MOF-guest structure configurations and their energies obtained with low parameters.

In [1]:
from ase import Atoms, Atom
from ase.visualize import view
from ase.io import read, write
from ase.build import mx2, add_adsorbate
from ase.constraints import FixAtoms
from ase.build import surface
from ase.data.colors import jmol_colors
from IPython import display
from sklearn.metrics.pairwise import pairwise_distances_argmin, pairwise_distances_argmin_min
from sklearn.ensemble import IsolationForest
from sklearn.manifold import MDS
import numpy as np



import os
from ase.collections import g2
from ase.build import molecule

###scikitlearn
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.model_selection import train_test_split



##plot
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

In [ ]:
main = os.getcwd()

#directories = ['alcohol_bencilico', 'benceno', 'benzaldehido',
               #'CO', 'CO2', 'H2S']
directories = ['H2S']
for i in directories:
    dir = main +'/'+ i
    print(dir)
    os.chdir(dir)
    os.mkdir('par_geom_energia')  ### directory where the test structure will be saved
    os.chdir('output')
    df = pd.read_csv('descriptors.csv',  index_col=0)  ### this file containf minimum distances from every valid structure
    
    ##Limpieza de datos para quitar outliers por medio de Multivariate Outlier Analysys: ###

    # Here we separate the independent variables for their analysis
    # ==============================================================================
    X = df[['Energy', 'd_O-Sc0', 'd_O-Sc1', 'd_O-Sc2', 'd_O-Sc3']]

    # Definition and training of the IsolationForest Model for Anomaly detection
    # Pleasenote that this is a unsupervised model and hence thereis no objective way to train it
    # The following is a Naive set of parammeters
    # ==============================================================================
    modelo_isof = IsolationForest(
                n_estimators  = 1000,
                max_samples   ='auto',
                contamination = 0.1,
                random_state  = 0)

    modelo_isof.fit(X)


    # Prediction from the Anomaly Detection Model
    # ==============================================================================
    X['anomaly'] = modelo_isof.predict(X)    # Anomaly prediction| 1:Ok | -1:Anomaly


    df_clean = df.loc[X['anomaly']==1]
    df_clean_index = list(df_clean.index)


    ####Normaling dataframe columns ###
    min_max_scaler = preprocessing.MinMaxScaler() #normalizing data
    mat = min_max_scaler.fit_transform(df_clean[['Energy', 'd_O-Sc0', 'd_O-Sc1', 'd_O-Sc2', 'd_O-Sc3']])
    #comb = list(combinations([0,1,2,3],2))
    df_normalize = pd.DataFrame(mat)


    # Reset index to ignore old indexes 
    df1_selected = df_clean['No. Structure'].reset_index(drop=True)
    df2_selected = df_normalize.reset_index(drop=True)
    result = pd.concat([df1_selected, df2_selected], axis=1)
    result = result.rename(columns={0: 'Energy', 1: 'd_O-Sc0', 2: 'd_O-Sc1', 3: 'd_O-Sc2', 4: 'd_O-Sc3'})


    ##Starts clustering with normalized data ####
    colors = ['#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00']
    lowenergy = result.sort_values(by=['Energy'])
    ind = lowenergy.index.values


    #MDS Method 
    mds = MDS(n_components=2,
                random_state=0,
                metric=True,
                n_init=4,
                max_iter=300,
                verbose=0,
                eps=0.001,
                dissimilarity='euclidean',
                n_jobs=1)
    pos = mds.fit(df_normalize).embedding_

    clusters = [3]
    for nc in clusters:
        km = KMeans(n_clusters=nc,
                       init='k-means++',
                       n_init=10,
                       max_iter=300,
                       random_state=0)
        km.fit(df_normalize)

        # We want to have the same colors for the same cluster from the MiniBatchKMeans and the KMeans algorithm. Let's pair the cluster centers per closest one.
        k_means_cluster_centers = km.cluster_centers_  #centros de los clusters 
        k_means_labels = pairwise_distances_argmin(df_normalize, k_means_cluster_centers) #Compute minimum distances between one point and a set of points
        closest, dist = pairwise_distances_argmin_min(k_means_cluster_centers, df_normalize) #return data indexes closest to centroides and the distances
        print('Los indices de los puntos más cercanos al centroidese de cada cluster son: ', closest)
        a = k_means_labels.tolist()


        #Plotting clusters 
    fig =  plt.figure(figsize=(10, 8))
    for k, col in zip(range(nc), colors):
            my_members = k_means_labels == k
             #cluster_center = pos[-nc:,:][k]
            plt.scatter(pos[my_members, 0], pos[my_members, 1], c=col, s=16, label=k+1, edgecolor='none', alpha=0.4)
            plt.scatter(pos[my_members, 0].mean(), pos[my_members, 1].mean(), c=col, marker='*', s=150, edgecolor='k')
    #plt.scatter(pos[ind[0], 0], pos[ind[0], 1], c='k', s=40, label='Energy')
    plt.xticks(())
    plt.yticks(())
    plt.savefig('clusters_{}.png'.format(i)) ## se guardan graficas en output

    # copying structurs closest to the centroid of every cluster to our directory.
    for j in closest:
        str_close_centro = result.iloc[j]
        os.system('cp -rf conf_{} ../par_geom_energia'.format(int(str_close_centro['No. Structure'])))

    os.chdir('../../')